
# Generative Adversarial Network: Toronto Face Likelihood Experiments

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from utils import save_weights, load_weights
import scipy.io
from torch import nn


from torch.utils.data import TensorDataset, DataLoader

import torch
from torch.utils.data import DataLoader, TensorDataset

from src.gan import GAN

from sklearn.model_selection import train_test_split

from likelihood import cross_validate_sigma, estimate_log_likelihood
from utils import compute_mean_std, normalize_data, rescale_to_unit_interval_individual, rescale_to_unit_interval_global, save_weights

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data = scipy.io.loadmat('./data/TFD/TFD_48x48.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'images', 'labs_ex', 'labs_id', 'folds'])

In [4]:
def configure_tfd(data, device='cuda', batch_size=100):
    images = data['images']
    labels = data['labs_id']

    # Convert images to torch tensor if needed
    if not isinstance(images, torch.Tensor):
        images = torch.tensor(images, dtype=torch.float32)
    images = images.to(device)

    print(f"Total number of samples: {images.shape[0]}")

    # Flatten images
    X = images.reshape(images.shape[0], -1)

    # Shuffle and split indices BEFORE normalization
    total_samples = X.shape[0]
    perm = torch.randperm(total_samples, device=device)
    train_size = int(0.85 * total_samples)
    train_idx = perm[:train_size]
    test_idx = perm[train_size:]

    X_train_raw = X[train_idx]
    X_test = X[test_idx]  # leave unprocessed

    # Compute mean and std from training set only
    mean = X_train_raw.mean(dim=0)
    std = X_train_raw.std(dim=0)

    # Normalize only training data
    X_train = (X_train_raw - mean) / std

    # Create DataLoader for normalized training data
    train_dataset = TensorDataset(X_train, torch.zeros(X_train.size(0), dtype=torch.uint8, device=device))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    return train_loader, X_train, X_test, mean, std

In [5]:
train_loader, X_train, X_test, mean, std = configure_tfd(data, device=device, batch_size=100)

Total number of samples: 102236


### Paper Configuration

In [6]:
LATENT_DIM = 100
NUM_EPOCHS = 300
UNIFORM_RANGE = 3 ** 0.5
GENERATOR_HIDDEN_DIM=8000
DISCRIMINATOR_HIDDEN_DIM=1200
INPUT_DIM = 2304
LR = 0.05
MIN_LR = 1e-6
DECAY_FACTOR = (1/(1+4e-6))
MOMENTUM = 0.5
FINAL_MOMENTUM = 0.7
MOMENTUM_SATURATE = 250
BATCH_SIZE = 100
USE_SIGMOID_GEN = False



In [7]:
gan = GAN(
    latent_dim=LATENT_DIM,
    input_size=INPUT_DIM,
    generator_hidden_dim=GENERATOR_HIDDEN_DIM,
    discriminator_hidden_dim=DISCRIMINATOR_HIDDEN_DIM,
    use_sigmoid_gen=USE_SIGMOID_GEN,
    device=device
)



In [8]:
gan.train_mbgd(
    data_loader=train_loader,
    learning_rate=LR,
    uniform_range=UNIFORM_RANGE,
    min_lr=MIN_LR,
    decay_factor=DECAY_FACTOR,
    epochs=NUM_EPOCHS,
    momentum=MOMENTUM,
    final_momentum=FINAL_MOMENTUM,
    momentum_saturate=MOMENTUM_SATURATE,
    log_dir='./tmp_runs'
)

[Epoch 1/300] Batch 0 | G Loss: 0.6951 | D Loss: 1.3865 | LR: 0.050000 | Momentum: 0.5000
[Epoch 1/300] Batch 100 | G Loss: 2.6417 | D Loss: 0.1725 | LR: 0.049980 | Momentum: 0.5000
[Epoch 1/300] Batch 200 | G Loss: 5.6592 | D Loss: 0.5051 | LR: 0.049960 | Momentum: 0.5000
[Epoch 1/300] Batch 300 | G Loss: 2.5310 | D Loss: 0.5150 | LR: 0.049940 | Momentum: 0.5000
[Epoch 1/300] Batch 400 | G Loss: 1.9004 | D Loss: 0.4049 | LR: 0.049920 | Momentum: 0.5000
[Epoch 1/300] Batch 500 | G Loss: 1.9381 | D Loss: 0.4915 | LR: 0.049900 | Momentum: 0.5000
[Epoch 1/300] Batch 600 | G Loss: 2.5909 | D Loss: 0.7566 | LR: 0.049880 | Momentum: 0.5000
[Epoch 1/300] Batch 700 | G Loss: 2.7229 | D Loss: 0.4005 | LR: 0.049860 | Momentum: 0.5000
[Epoch 1/300] Batch 800 | G Loss: 3.0616 | D Loss: 0.5540 | LR: 0.049840 | Momentum: 0.5000
[Epoch 2/300] Batch 0 | G Loss: 3.3330 | D Loss: 0.3974 | LR: 0.049827 | Momentum: 0.5000
[Epoch 2/300] Batch 100 | G Loss: 4.3688 | D Loss: 0.8840 | LR: 0.049807 | Momentum:

In [8]:
gan.load_weights(path_prefix="300_tfd_gan_weights", device=device)

Weights loaded from 300_tfd_gan_weights_*.pth


In [9]:
samples = gan.sample_z(batch_size=100000, uniform_range=UNIFORM_RANGE)
samples = gan.generator(samples).detach()

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.98 GiB. GPU 0 has a total capacity of 7.91 GiB of which 257.69 MiB is free. Including non-PyTorch memory, this process has 7.45 GiB memory in use. Of the allocated memory 7.27 GiB is allocated by PyTorch, and 59.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [11]:
cross_validate_sigma(
    samples=rescale_to_unit_interval_individual(samples, mean, std),
    validation_dataset=rescale_to_unit_interval_individual(X_train[0:10000], mean, std),
    sigma_range=np.exp(np.linspace(np.log(0.09), np.log(0.5),40)),
    batch_size=100,
)

Evaluating sigma = 0.08999999999999998
Sigma: 0.09000, Log-Likelihood: 1769.41218
Evaluating sigma = 0.09404551439665289
Sigma: 0.09405, Log-Likelihood: 1818.78718
Evaluating sigma = 0.0982728753125672
Sigma: 0.09827, Log-Likelihood: 1885.94755
Evaluating sigma = 0.10269025677787244
Sigma: 0.10269, Log-Likelihood: 1888.07175
Evaluating sigma = 0.10730620024665986
Sigma: 0.10731, Log-Likelihood: 1887.96365
Evaluating sigma = 0.11212963111274855
Sigma: 0.11213, Log-Likelihood: 1918.23878
Evaluating sigma = 0.1171698759678375
Sigma: 0.11717, Log-Likelihood: 1886.46478
Evaluating sigma = 0.12243668063541442
Sigma: 0.12244, Log-Likelihood: 1880.03388
Evaluating sigma = 0.12794022901529187
Sigma: 0.12794, Log-Likelihood: 1859.38887
Evaluating sigma = 0.1336911627752078
Sigma: 0.13369, Log-Likelihood: 1824.89152
Evaluating sigma = 0.13970060192756747
Sigma: 0.13970, Log-Likelihood: 1786.50862
Evaluating sigma = 0.1459801663311125
Sigma: 0.14598, Log-Likelihood: 1733.03859
Evaluating sigma = 0

np.float64(0.11212963111274855)

In [12]:
# 0.11212963111274855
estimate_log_likelihood(
    samples=rescale_to_unit_interval_individual(samples, mean, std),
    test_data=rescale_to_unit_interval_global(X_test),
    sigma=0.11104586060596347

)


(np.float64(2162.721767602233), np.float64(7.311285554510226))